In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [1]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir

from geo_tool import Point_Cloud

from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.convenience import reconstruct_pclouds

from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats
from tf_lab.evaluate.generative_pc_nets import entropy_of_occupancy_grid, jensen_shannon_divergence, minimum_mathing_distance

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
def identity(x):
    return x

pc_normalizer = identity

In [4]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
experiment_tag =  'mlp_with_split_1pc_usampled_bnorm_on_encoder_only'
n_pc_points = 2048
random_seed = 42
ae_loss = 'chamfer'
class_name = raw_input('Give me the class type: ').lower()
syn_id = snc_category_to_synth_id()[class_name]
b_neck = 128
voxel_resolution = 28
cmp_in_sphere = True

Give me the class type: chair


In [5]:
# Prepare GT for JSD comparisons
in_data = load_multiple_version_of_pcs('uniform_one', syn_id, n_classes=1)

tr_pc = in_data['train'].point_clouds
te_pc = in_data['test'].point_clouds
va_pc = in_data['val'].point_clouds

train_data = pc_normalizer(tr_pc)
test_data = pc_normalizer(te_pc) 
val_data = pc_normalizer(va_pc)

Loading test data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt
679 pclouds were loaded. They belong in 1 shape-classes.
Loading train data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt
5761 pclouds were loaded. They belong in 1 shape-classes.
Loading val data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt
338 pclouds were loaded. They belong in 1 shape-classes.


In [7]:
_, train_grid_var = entropy_of_occupancy_grid(train_data, voxel_resolution, in_sphere=cmp_in_sphere)
_, test_grid_var = entropy_of_occupancy_grid(test_data, voxel_resolution, in_sphere=cmp_in_sphere)
_, val_grid_var = entropy_of_occupancy_grid(val_data, voxel_resolution, in_sphere=cmp_in_sphere)

def jsd_on_reconstructed_data(ae_model, pclouds, cmp_grid_var, voxel_resolution, cmp_in_sphere):
    recon, _ = reconstruct_pclouds(ae_model, pclouds, batch_size=100)
    recon = pc_normalizer(recon)
    _, recon_grid_var = entropy_of_occupancy_grid(recon, voxel_resolution, in_sphere=cmp_in_sphere)
    return jensen_shannon_divergence(recon_grid_var, cmp_grid_var)

In [6]:
experiment_id = '_'.join(['ae', class_name, experiment_tag, str(n_pc_points), 'pts', str(b_neck), 'bneck', ae_loss])
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', experiment_id)
conf = Conf.load(osp.join(train_dir, 'configuration'))    
val_error, best_epoch = find_best_validation_epoch_from_train_stats(osp.join(train_dir, 'train_stats.txt'))    
if best_epoch % conf.saver_step != 0: # Model was not saved at that epoch: use next bigger epoch.
    epoch_to_load = best_epoch + (best_epoch % conf.saver_step)
else:
    epoch_to_load = best_epoch    
conf.encoder_args['verbose'] = False
conf.decoder_args['verbose'] = False    
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)    
ae.restore_model(conf.train_dir, epoch_to_load, verbose=True)

Model restored in epoch 500.


In [30]:
j1 = jsd_on_reconstructed_data(ae, tr_pc, train_grid_var, voxel_resolution, cmp_in_sphere)
j2 = jsd_on_reconstructed_data(ae, va_pc, train_grid_var, voxel_resolution, cmp_in_sphere)
j3 = jsd_on_reconstructed_data(ae, te_pc, train_grid_var, voxel_resolution, cmp_in_sphere)

In [31]:
print j1, j2, j3

0.0216779896205 0.0305057765861 0.0256130541121


In [7]:
def sampling_mmd(sample_data, ref_data, n_samples=10, batch_size=100, emd=True):
    ids = np.arange(len(ref_data))
    scores = []
    for i in range(n_samples):
        print i
        sb = np.random.choice(ids, batch_size, replace=False)        
        mi = minimum_mathing_distance(sample_data[sb], ref_data[sb], batch_size, normalize=True, use_sqrt=True, use_EMD=emd)
        scores.append(mi[0])
    scores = np.array(scores)
    return scores

In [8]:
recon, _ = reconstruct_pclouds(ae, tr_pc, 100)

In [9]:
s = sampling_mmd(recon, tr_pc, emd=False)
print np.mean(s), np.std(s)

0
1
2
3
4
5
6
7
8
9
0.0258957 0.000257555


In [13]:
# mi = minimum_mathing_distance(recon, tr_pc, 6000, normalize=True, use_sqrt=True)[0]
print mi

0.0259442
